In [49]:
%pip install "drawsvg~=2.0"

import csv
import itertools
import random
import os
import drawsvg as dw

Note: you may need to restart the kernel to use updated packages.


## Define sampling params here

In [50]:
sample_size = 50
angle_variation_max = 20 # Pixel value, not degrees
fixed_vars = ["line_color"]

w, h = 600, 600

save_path = "../jnd_images_svg/angle"
os.makedirs(save_path, exist_ok=True)

## Define hyperparam space

In [51]:
hparam_dict = {
    "line_color": ["red", "green", "blue"],
    "line_thickness": range(1, 4),
    "line_location": [0.25, 0.5, 0.75],
    "line_dist": [0.1, 0.2, 0.3],
    "reference_angle": [0, 10, 20],   # note that this doesn't correspond to geometric angles but 1px distance
    "reference_line_length": [50, 100, 150],
}

selected_hparams = [key for key in hparam_dict if key not in fixed_vars]
selected_hparam_vals = [hparam_dict[key] for key in selected_hparams]
fixed_hparam_vals = [hparam_dict[key] for key in fixed_vars]
print(selected_hparams)

all_hparams = list(itertools.product(*selected_hparam_vals))
fixed_hparams_exhaustive = list(itertools.product(*fixed_hparam_vals))

print(len(all_hparams))
print(len(fixed_hparams_exhaustive))

['line_thickness', 'line_location', 'line_dist', 'reference_angle', 'reference_line_length']
243
3


## Sample

In [52]:
random_hparam_configs = random.choices(all_hparams, k=sample_size)

## Generate & save

In [54]:
oob_count = 0
total_count = 0
param_d_list = []

# Iterate over randomly selected hparam configurations minus the fixed hparam
for i, config in enumerate(random_hparam_configs):
    config_d = dict(zip(selected_hparams, config))
    
    # Iterate over all values of a fixed variable
    for j, fixed_config in enumerate(fixed_hparams_exhaustive):
        config_d.update(dict(zip(fixed_vars, fixed_config)))
        
        # Iterate over target manipulation (angle)
        for k, angle in enumerate(range(0, angle_variation_max)):
            location = config_d["line_location"]
            ref_len = config_d["reference_line_length"]
            ref_angle = config_d["reference_angle"]
            line_dist = config_d["line_dist"]
            color = config_d["line_color"]
            thickness = config_d["line_thickness"]
            
            ref_x1, ref_y1 = w*location, h*location
            ref_x2, ref_y2 = w*location+ref_len, h*location+ref_angle

            x1, y1 = w*location, h*(location+line_dist)
            x2, y2 = w*location + ref_len, h*(location+line_dist)+ref_angle+angle

            # Count cases where lines go out of bounds
            total_count += 1
            if ref_x1 > w or ref_x2 > w or ref_y1 > h or ref_y2 > h:
                print(ref_x1, ref_x2, ref_y1, ref_y2)
                oob_count += 1
                continue

            d = dw.Drawing(
                w, h, origin=(0, 0),
                context=None,
                animation_config=None,
                id_prefix='d'
            )
            line = dw.Line(
                ref_x1, ref_y1,
                ref_x2, ref_y2,
                stroke=color,
                stroke_width=thickness
            )
            line2 = dw.Line(
                x1, y1,
                x2, y2,
                stroke=color,
                stroke_width=thickness
            )
            d.append(line)
            d.append(line2)

#             display(d)
            idx = f"{i}_{j}_{k}"
            filename = f"{idx}.svg"
            d.save_svg(os.path.join(save_path, filename))
        
            param_d = {
                "idx": idx,
                "filename": filename,
                "l1_x1": ref_x1,
                "l1_y1": ref_y1,
                "l1_x2": ref_x2,
                "l1_y2": ref_y2,
                "l2_x1": x1,
                "l2_y1": y1,
                "l2_x2": x2,
                "l2_y2": y2,
                "line_color": color,
                "line_thickness": thickness,
                "line_location": location,
                "line_dist": line_dist,
                "reference_angle": ref_angle,
                "reference_line_length": ref_len,
                "pixel_diff_y": angle
            }

            param_d_list.append(param_d)

print("Total # images: ", total_count)
print("Total out of bound images: ", oob_count)

3000
0


## Save metadata

In [56]:
print(param_d_list[-1])

with open(os.path.join(save_path, "angle_jnd_images_svg.csv"), 'w') as wf:
    writer = csv.DictWriter(wf, fieldnames=param_d_list[-1].keys())
    writer.writeheader()
    for d in param_d_list:
        writer.writerow(d)

{'idx': '49_2_19', 'filename': '49_2_19.svg', 'l1_x1': 150.0, 'l1_y1': 150.0, 'l1_x2': 250.0, 'l1_y2': 160.0, 'l2_x1': 150.0, 'l2_y1': 330.0, 'l2_x2': 250.0, 'l2_y2': 359.0, 'line_color': 'blue', 'line_thickness': 3, 'line_location': 0.25, 'line_dist': 0.3, 'reference_angle': 10, 'reference_line_length': 100, 'pixel_diff_y': 19}
